In [ ]:
%cd ~
import pyfits,glob,time,scipy
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell

topdir='/Users/cslage/Research/LSST/code/GUI/'
thedir=topdir+'profiles/'
%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'

%matplotlib inline

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(numspots, nx, ny, sigmax, sigmay, Imax):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    for n in range(numspots):
        spotlist.xoffset[n] = -0.5 + rand()
        spotlist.yoffset[n] = -0.5 + rand()     
        for i in range(nx):
            for j in range(ny):
                noise = 0.0#Imax * 1.0E-2 * rand() 
                xl = spotlist.x[i] - spotlist.xoffset[n] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[j] - spotlist.yoffset[n] - 0.5
                yh = yl + 1.0
                spotlist.data[i,j,n] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax) + noise
    return spotlist

import forward

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
nx = 9
ny = 9
numspots = 100
Imax = 10000
sigmax = 1.0
sigmay = 1.0
    
spotlist = BuildSpotList(numspots, nx, ny, sigmax, sigmay, Imax)

print "nstamps = %d"%spotlist.nstamps

for spot in range(10):
    xsum = 0.0
    ysum = 0.0
    datasum = 0.0
    for i in range(nx):
        for j in range(ny):
            ysum += spotlist.y[j] * spotlist.data[i,j,spot]
            xsum += spotlist.x[i] * spotlist.data[i,j,spot]
            datasum += spotlist.data[i,j,spot]
    xoff = xsum / datasum
    yoff = ysum / datasum

    
    print "Original offset = (%.4f, %.4f), Calculated offset = (%.4f, %.4f)"%(spotlist.xoffset[spot],spotlist.yoffset[spot],xoff,yoff)

In [ ]:
nx = 9
ny = 9
numspots = 5000
Imax = 10000
sigma = .001

sigmain = []
sigmaout = []

for i in range(40):
    sigmax = sigma * 1.25**i
    sigmay = sigmax
    sigmain.append(sigmax)
    
    spotlist = BuildSpotList(numspots, nx, ny, sigmax, sigmay, Imax)

    param0 = [1.00, 1.00]

    args = ()#(spotlist)
    Result = fmin_powell(FOM, param0, args)
    sigmaout.append((abs(Result[0]) + abs(Result[1]))/2.0)
    print "Sigmain = %f, Sigmaout = %f"%(sigmain[-1],sigmaout[-1])


In [ ]:
figure()
title("Test of Forward Model Extraction - Noise = 1.0% Full Scale")
scatter(log10(sigmain),log10(sigmaout))
xplot=linspace(-3.25, 1.0 ,100)
yplot = xplot
plot(xplot, yplot, color='red')

xlabel('Log Input Sigma (pixels)')
ylabel('Log Modeled Sigma (pixels)')

savefig("Forward_Test_Noisy_2.png")


In [ ]:
figure()
#testn = 45
spot = 888
#spotlist = spotlists[testn]
#Result = results[testn]
Imax = spotlist.imax[spot]
sigmax = Result[0]
sigmay = Result[1]
#seqno = int(zfilelist[testn].split("_")[3])
#suptitle=("Sequence # %d, Spot # %d"%(seqno,spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)

subplots_adjust(wspace = 1.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot],interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot], lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
legend(loc = (-4.0,0.8))
savefig("Typical_Fit_Powell")
